In [1]:
import os
import numpy as np
import import_ipynb
import Project
np.random.seed(1234)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from utils import CharacterTable, transform
from utils import batch, datagen, decode_sequences
from utils import read_text, tokenize
from model import seq2seq
error_rate = 0.8
hidden_size = 512
nb_epochs = 100
train_batch_size = 128
val_batch_size = 256
sample_mode = 'argmax'
reverse = True
data_path = r'C:\Users\ganes\Music\data'
train_books = ['nietzsche.txt', 'pride_and_prejudice.txt',
               'shakespeare.txt', 'war_and_peace.txt']
val_books = ['wonderland.txt']
if __name__ == '__main__':
    # Prepare training data.
    text  = read_text(data_path, train_books)
    vocab = tokenize(text)
    vocab = list(filter(None, set(vocab)))
    maxlen = max([len(token) for token in vocab]) + 2
    train_encoder, train_decoder, train_target = transform(
        vocab, maxlen, error_rate=error_rate, shuffle=False)
    print(train_encoder[:10])
    print(train_decoder[:10])
    print(train_target[:10])

    input_chars = set(' '.join(train_encoder))
    target_chars = set(' '.join(train_decoder))
    nb_input_chars = len(input_chars)
    nb_target_chars = len(target_chars)

    print('Size of training vocabulary =', len(vocab))
    print('Number of unique input characters:', nb_input_chars)
    print('Number of unique target characters:', nb_target_chars)
    print('Max sequence length in the training set:', maxlen)

    # Prepare validation data.
    text = read_text(data_path, val_books)
    val_tokens = tokenize(text)
    val_tokens = list(filter(None, val_tokens))

    val_maxlen = max([len(token) for token in val_tokens]) + 2
    val_encoder, val_decoder, val_target = transform(
        val_tokens, maxlen, error_rate=error_rate, shuffle=False)
    print(val_encoder[:10])
    print(val_decoder[:10])
    print(val_target[:10])
    print('Number of non-unique validation tokens =', len(val_tokens))
    print('Max sequence length in the validation set:', val_maxlen)

    # Define training and evaluation configuration.
    input_ctable  = CharacterTable(input_chars)
    target_ctable = CharacterTable(target_chars)

    train_steps = len(vocab) // train_batch_size
    val_steps = len(val_tokens) // val_batch_size

    # Compile the model.
    model, encoder_model, decoder_model = seq2seq(
        hidden_size, nb_input_chars, nb_target_chars)
    print(model.summary())

    # Train and evaluate.
    for epoch in range(nb_epochs):
        print('Main Epoch {:d}/{:d}'.format(epoch + 1, nb_epochs))
    
        train_encoder, train_decoder, train_target = transform(
            vocab, maxlen, error_rate=error_rate, shuffle=True)
        
        train_encoder_batch = batch(train_encoder, maxlen, input_ctable,
                                    train_batch_size, reverse)
        train_decoder_batch = batch(train_decoder, maxlen, target_ctable,
                                    train_batch_size)
        train_target_batch  = batch(train_target, maxlen, target_ctable,
                                    train_batch_size)    

        val_encoder_batch = batch(val_encoder, maxlen, input_ctable,
                                  val_batch_size, reverse)
        val_decoder_batch = batch(val_decoder, maxlen, target_ctable,
                                  val_batch_size)
        val_target_batch  = batch(val_target, maxlen, target_ctable,
                                  val_batch_size)
    
        train_loader = datagen(train_encoder_batch,
                               train_decoder_batch, train_target_batch)
        val_loader = datagen(val_encoder_batch,
                             val_decoder_batch, val_target_batch)
    
        model.fit(train_loader,steps_per_epoch=train_steps,epochs=1, verbose=1,validation_data=val_loader,validation_steps=val_steps)
        # On epoch end - decode a batch of misspelled tokens from the validation set to visualize speller performance.
        nb_tokens = 5
        input_tokens, target_tokens, decoded_tokens = decode_sequences(
            val_encoder, val_target, input_ctable, target_ctable,
            maxlen, reverse, encoder_model, decoder_model, nb_tokens,
            sample_mode=sample_mode, random=True)
        
        print('-')
        print('Input tokens:  ', input_tokens)
        print('Decoded tokens:', decoded_tokens)
        print('Target tokens: ', target_tokens)
        print('-')
        
        # Save the model at end of each epoch.
        model_file = '_'.join(['seq2seq', 'epoch', str(epoch + 1)]) + '.h5'
        save_dir = 'checkpoints'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
        save_path = os.path.join(save_dir, model_file)
        print('Saving full model to {:s}'.format(save_path))
        model.save(save_path)


importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from model.ipynb
['semm******************', 'Ynoder****************', 'roepened**************', 'unforeeen*************', "ress'd****************", "jAdversity's**********", 'grants****************', 'avWnging**************', 'forevIer**************', 'tunc******************']
['\tseem*****************', '\tYonder***************', '\treopened*************', '\tunforeseen***********', "\tpress'd**************", "\tAdversity's**********", '\tgrants***************', '\tavenging*************', '\tforever**************', '\ttunic****************']
['seem******************', 'Yonder****************', 'reopened**************', 'unforeseen************', "press'd***************", "Adversity's***********", 'grants****************', 'avenging**************', 'forever***************', 'tunic*****************']
Size of training vocabulary = 33047
Number of unique input characters: 55
Number of unique target characters: 56
Max se

-
Input tokens:   ['snathc', 'stUill', 'of', 'me', 'satid']
Decoded tokens: ['snatch', 'still', 'of', 'me', 'satid']
Target tokens:  ['snatch', 'still', 'of', 'me', 'said']
-
Saving full model to checkpoints\seq2seq_epoch_25.h5
Main Epoch 26/100
Shuffling data.
258/258 [==============================] - 593s 2s/step - loss: 0.1176 - accuracy: 0.9678 - truncated_acc: 0.9557 - truncated_loss: 0.1618 - val_loss: 0.0949 - val_accuracy: 0.9760 - val_truncated_acc: 0.9670 - val_truncated_loss: 0.1304
-
Input tokens:   ['liRe', 'as', 'to', 'fijnd', 'tPe']
Decoded tokens: ['lime', 'as', 'to', 'fined', 'tee']
Target tokens:  ['like', 'as', 'to', 'find', 'the']
-
Saving full model to checkpoints\seq2seq_epoch_26.h5
Main Epoch 27/100
Shuffling data.
258/258 [==============================] - 575s 2s/step - loss: 0.1151 - accuracy: 0.9685 - truncated_acc: 0.9566 - truncated_loss: 0.1582 - val_loss: 0.0908 - val_accuracy: 0.9777 - val_truncated_acc: 0.9693 - val_truncated_loss: 0.1248
-
Input token

258/258 [==============================] - 560s 2s/step - loss: 0.0894 - accuracy: 0.9754 - truncated_acc: 0.9662 - truncated_loss: 0.1229 - val_loss: 0.0802 - val_accuracy: 0.9794 - val_truncated_acc: 0.9717 - val_truncated_loss: 0.1102
-
Input tokens:   ['sZe', 'antd', "'I", 'saif', 'gnder']
Decoded tokens: ['se', 'anted', "'I", 'sail', 'gonder']
Target tokens:  ['see', 'and', "'I", 'said', 'under']
-
Saving full model to checkpoints\seq2seq_epoch_42.h5
Main Epoch 43/100
Shuffling data.
258/258 [==============================] - 561s 2s/step - loss: 0.0875 - accuracy: 0.9758 - truncated_acc: 0.9667 - truncated_loss: 0.1203 - val_loss: 0.0807 - val_accuracy: 0.9788 - val_truncated_acc: 0.9709 - val_truncated_loss: 0.1110
-
Input tokens:   ['frying', 'rst', 'a', 'cotent', 'anoqther']
Decoded tokens: ['frying', 'rust', 'a', 'content', 'another']
Target tokens:  ['frying', 'rest', 'a', 'content', 'another']
-
Saving full model to checkpoints\seq2seq_epoch_43.h5
Main Epoch 44/100
Shufflin

-
Input tokens:   ['he', 'ought', 'throgh', 'einto', 'was']
Decoded tokens: ['he', 'ought', 'through', 'ention', 'was']
Target tokens:  ['the', 'ought', 'through', 'into', 'was']
-
Saving full model to checkpoints\seq2seq_epoch_58.h5
Main Epoch 59/100
Shuffling data.
258/258 [==============================] - 557s 2s/step - loss: 0.0740 - accuracy: 0.9793 - truncated_acc: 0.9715 - truncated_loss: 0.1017 - val_loss: 0.0752 - val_accuracy: 0.9802 - val_truncated_acc: 0.9728 - val_truncated_loss: 0.1034
-
Input tokens:   ['if', 'ou', 'in', 'anixous', 'instead']
Decoded tokens: ['if', 'ou', 'in', 'anxious', 'instead']
Target tokens:  ['if', 'you', 'in', 'anxious', 'instead']
-
Saving full model to checkpoints\seq2seq_epoch_59.h5
Main Epoch 60/100
Shuffling data.
258/258 [==============================] - 557s 2s/step - loss: 0.0738 - accuracy: 0.9795 - truncated_acc: 0.9718 - truncated_loss: 0.1014 - val_loss: 0.0727 - val_accuracy: 0.9813 - val_truncated_acc: 0.9743 - val_truncated_loss: 

258/258 [==============================] - 591s 2s/step - loss: 0.0630 - accuracy: 0.9823 - truncated_acc: 0.9756 - truncated_loss: 0.0866 - val_loss: 0.0731 - val_accuracy: 0.9811 - val_truncated_acc: 0.9740 - val_truncated_loss: 0.1006
-
Input tokens:   ['gettSng', 'ssee', 'Vhat', 'ytell', 'to']
Decoded tokens: ['getting', 'sees', 'What', 'yell', 'to']
Target tokens:  ['getting', 'see', 'that', 'tell', 'to']
-
Saving full model to checkpoints\seq2seq_epoch_75.h5
Main Epoch 76/100
Shuffling data.
258/258 [==============================] - 590s 2s/step - loss: 0.0643 - accuracy: 0.9820 - truncated_acc: 0.9752 - truncated_loss: 0.0884 - val_loss: 0.0696 - val_accuracy: 0.9817 - val_truncated_acc: 0.9748 - val_truncated_loss: 0.0957
-
Input tokens:   ['exactly', 'Aay', 'a', 'Mthe', 'hoem']
Decoded tokens: ['exactly', 'Aay', 'a', 'Mothe', 'home']
Target tokens:  ['exactly', 'way', 'a', 'the', 'home']
-
Saving full model to checkpoints\seq2seq_epoch_76.h5
Main Epoch 77/100
Shuffling data.


-
Input tokens:   ['Peat', 'hVm', 'in', 'my', 'it']
Decoded tokens: ['Peat', 'him', 'in', 'my', 'it']
Target tokens:  ['beat', 'him', 'in', 'my', 'it']
-
Saving full model to checkpoints\seq2seq_epoch_91.h5
Main Epoch 92/100
Shuffling data.
258/258 [==============================] - 565s 2s/step - loss: 0.0583 - accuracy: 0.9834 - truncated_acc: 0.9771 - truncated_loss: 0.0801 - val_loss: 0.0704 - val_accuracy: 0.9818 - val_truncated_acc: 0.9750 - val_truncated_loss: 0.0968
-
Input tokens:   ['th', 'well', 'Turtle', 'sHt', 'in']
Decoded tokens: ['th', 'well', 'Turtle', 'set', 'in']
Target tokens:  ['the', 'well', 'Turtle', 'set', 'in']
-
Saving full model to checkpoints\seq2seq_epoch_92.h5
Main Epoch 93/100
Shuffling data.
258/258 [==============================] - 568s 2s/step - loss: 0.0574 - accuracy: 0.9837 - truncated_acc: 0.9775 - truncated_loss: 0.0789 - val_loss: 0.0716 - val_accuracy: 0.9818 - val_truncated_acc: 0.9749 - val_truncated_loss: 0.0985
-
Input tokens:   ['fit', 'ou